In [2]:
import os
os.chdir(os.path.dirname(os.getcwd()))
from envs import SimpleScheduler, DCScheduler
from tools import Logger

## 虚拟影院环境介绍
### （一）简单影院经营环境
该环境以2019年3月1日至2024年9月30日上映的电影作为数据基础，包含了每一部电影在每一天的经营数据，包括“当前票房”， “当前场次”，“当前人次”，“票房占比”，“场次占比”，“当前平均票价”，“场均人次”，“场均收入”，“周索引”这些信息。

在该环境中，规定了：
- 同时上映的电影数量为6部
- 用于决策的影片经营数据为最近7天的经营数据
- 每天能够上映的总场次为100场
- 总的测试期为40天
#### Reset
该环境进行重置（reset）时，可通过`fix_start_date`参数来说指定特定的开始日期。如果设置该参数，那么就从固定的日期开始测试，并且每一日上线的影片也是固定的；如果没有设置该参数，那么就从随机的日期开始测试，并且每一日上线的影片也是随机的。当随机选择日期时，如果选择的日期在数据集中对应的影片数量不足6部，那么就会向后推一天，直到找到有足够数量影片的日期为止。

确定日期和影片名称之后，Reset函数会返回截止该日期，这些影片在过去七天的经营数据。

测试代码如下：


In [3]:
logger = Logger(logdir='logs/jupyter_log.txt')
env = SimpleScheduler(logger)
feature_array = env.reset(fix_start_date='2022-10-01')

[INFO] [2025-07-05 15:26:12] 开始重置环境
[INFO] [2025-07-05 15:26:12] 使用指定的日期进行重置：2022-10-01
[INFO] [2025-07-05 15:26:12] ++++++++++++++++++++ [2022-10-01 00:00:00 周6] ++++++++++++++++++++++++
[INFO] [2025-07-05 15:26:12] 明日可播放电影名称为：《我的英雄》, 《人间世》, 《新神榜：杨戬》, 《我的非凡父母》, 《哥，你好》, 《世间有她》


`feature_array`以numpy数组的形式返回，包含了6部影片在过去7天的16个经营数据。数据的形状为`(6, 7, 16)`，其中6表示影片数量，7表示过去7天的经营数据，16表示经营数据的维度。具体的维度如下：
- 0: 当前票房，以万元为单位，表示该电影在全国市场上本日的总票房收入。
- 1: 当前场次，表示该电影在全国市场上本日的总放映场次。
- 2: 当前人次，以万人为单位，表示该电影在全国市场上本日的观影人数。
- 3: 票房占比，表示该电影在全国市场上本日的总票房收入占全国市场总票房收入的比例。
- 4: 场次占比，表示该电影在全国市场上本日的总放映场次占全国市场总放映场次的比例。
- 5: 人次占比，表示该电影在全国市场上本日的观影人数占全国市场观影人数的比例。
- 6: 当前平均票价，表示该电影在全国市场上本日的平均票价。
- 7: 场均人次，表示该电影在全国市场上本日的平均观影人数。
- 8: 场均收入，表示该电影在全国市场上本日的平均票房收入，即观影人数乘以平均票价。
- 9~15：为周索引，长度为7的向量，比如周一的向量为[1, 0, 0, 0, 0, 0, 0]，周二的向量为[0, 1, 0, 0, 0, 0, 0]，以此类推。

下面是一个例子，展示了某一部影片在国庆7天的经营数据（不包括周索引）。


In [3]:
import pandas as pd
pd.DataFrame(feature_array[2, : , :9], columns=['当前票房(万)', '当前场次', '当前人次(万)', '票房占比', '场次占比', '人次占比',
                                '当前平均票价', '场均人次', '场均收入'])

,当前票房(万),当前场次,当前人次(万),票房占比,场次占比,人次占比,当前平均票价,场均人次,场均收入
0,714.250783,41524.0,16.8967,15.86,12.84,15.05,42.0,4.0,172.0
1,139.912492,33128.0,3.3196,9.03,12.03,8.53,42.0,1.0,42.0
2,132.656946,32842.0,3.1368,8.90,12.03,8.31,42.0,1.0,40.0
3,129.748353,32850.0,3.0799,7.19,12.05,6.97,42.0,1.0,39.0
4,127.812619,31763.0,3.0519,4.69,11.77,4.72,42.0,1.0,40.0
5,191.954154,17500.0,4.5130,2.23,5.52,2.17,43.0,3.0,110.0
6,568.980988,7662.0,13.3947,2.13,1.81,2.07,42.0,17.0,743.0


#### Step
决策模型需要根据feature_array，生成一个6维的动作向量，这个动作向量表示六部正在上映的影片在未来一天的放映场次占比。因此这个动作向量应该满足（1）每一个数值都不小于0，即不能够设定负数的播放场次；（2）全部的数值之和等于1。将这个动作向量输入给env.step方法之后，就可以获取到相应的：
（1）截止当日近7天的影片经营数据；
（2）当日总收入；
（3）测试结束标志；
（4）附带的详细经营数据（包括：截止目前的总收入，明日放映的影片名称列表，每一部电影当日收入列表，每一部电影当日场均收入列表）

测试代码如下：

In [4]:
import numpy as np
action = np.array([0.2, 0.2, 0.2, 0.2, 0.1, 0.1])
feature_array, day_income, done, info = env.step(action)
info

[INFO] [2025-07-05 16:14:33] ++++++++++++++++++++ [2022-10-02 周6] ++++++++++++++++++++++++
[INFO] [2025-07-05 16:14:33] 电影名称：        我的英雄        ，场次：20 ，单场均收入：659  ，总收入：13180  
[INFO] [2025-07-05 16:14:33] 电影名称：        人间世         ，场次：20 ，单场均收入：0    ，总收入：0      
[INFO] [2025-07-05 16:14:33] 电影名称：       新神榜：杨戬       ，场次：20 ，单场均收入：574  ，总收入：11480  
[INFO] [2025-07-05 16:14:33] 电影名称：       我的非凡父母       ，场次：20 ，单场均收入：8    ，总收入：160    
[INFO] [2025-07-05 16:14:33] 电影名称：        哥，你好        ，场次：10 ，单场均收入：484  ，总收入：4840   
[INFO] [2025-07-05 16:14:33] 电影名称：        世间有她        ，场次：10 ，单场均收入：127  ，总收入：1270   
[INFO] [2025-07-05 16:14:33] 截止目前的 总收入：30930，总效用：27129.1
[INFO] [2025-07-05 16:14:33] 明日可播放电影名称为：《我的英雄》, 《人间世》, 《新神榜：杨戬》, 《我的非凡父母》, 《哥，你好》, 《世间有她》


{'TotalIncome': 30930,
 'MovieNames': ['我的英雄', '人间世', '新神榜：杨戬', '我的非凡父母', '哥，你好', '世间有她'],
 'MovieDayIncome': [13180, 0, 11480, 160, 4840, 1270],
 'MoviePerShowIncome': [659, 0, 574, 8, 484, 127]}

决策模型的目标是依据不同电影的经营数据，制定出最优的排片比率，从而在测试期内实现最大的经营收入。一种简单的人工智能方法就是利用时序神经网络（LSTM）来预测未来一天的经营数据，然后根据预测的不同电影的场均收入来制定最优的排片比率。请参考`policys.policy.RLScheduling`类所定义的方法，该方法与其他三种经验性排片对比，测试得分如下：

| 策略 |   评分   |
| :--- |:------:|
| RLScheduling | 368.2万 |
| 固定排片模式| 134.8万|
| 排片-票房占比滚动优化模式| 278.2|
| 前日场均收入最高模式| 325.0|

该环境后续可以尝试使用分布学习的方法来进行优化。电影的场均收入是一个典型的随机变量，服从某一个分布。学习出不同电影场均收入所服从的分布，就可以（1）更快的学习到最优经营策略（2）更好的管理经营风险。

### （二）选片+排片影院经营模式
在第一种环境中，影院的决策者不需要考虑播放哪一部片子，只需要根据过往的经营数据来决定为每一部影片分配多少的场次即可。为了更贴近实际的影院经营模式，第二种环境中，决策者需要同时考虑选片和排片的问题。选片是指在每一天可选的12部影片中，选择出6部影片进行放映；排片是指在这6部影片中，决定每一部影片的放映场次占比。
在选片的过程中，有两个要点：
（1）选片的过程中除了考虑影片的经营数据之外，还需要考虑影片的类型、时长、导演、演员等信息。这些信息可以从互联网上获取到。将这些非格式化信息与经营数据这类格式化信息结合起来，是一个新的技术尝试。
（2）如果不上映某一影片，那么该影片的经营数据就不能够获取到。这意味着探索太多就无法对影片的盈利能力进行充分的估计，探索太少就无法找到最有价值的影片，这也就是探索与贪心的平衡问题。

#### Reset
在该模式下，对环境进行重置之后，会得到两个数据，第一个是明日可供选择的12影片名称，第二个是截止当日的经营数据。该经营数据包含了15部影片在过去7天的经营数据。当然由于是重置环境，所以全部的经营数据都是0值。
测试代码如下：

In [5]:
logger = Logger(logdir='logs/jupyter_log.txt')
env = DCScheduler(logger)
candidate_movie_list, feature_map = env.reset(fix_start_date='2022-10-01')

[INFO] [2025-05-09 22:35:35] 开始重置环境
[INFO] [2025-05-09 22:35:35] 使用指定的日期进行重置：2022-10-01
[INFO] [2025-05-09 22:35:35] ++++++++++++++++++++ [2022-10-01 00:00:00 周6] ++++++++++++++++++++++++
[INFO] [2025-05-09 22:35:35] 明日可播放电影名称为：《我的英雄》, 《人间世》, 《新神榜：杨戬》, 《我的非凡父母》, 《哥，你好》, 《世间有她》, 《追光万里》, 《还是觉得你最好》, 《海的尽头是草原》, 《狼群》, 《妈妈！》, 《青蛙王国-极限运动》, 《追光》, 《钢铁意志》, 《平凡英雄》


In [6]:
candidate_movie_list

['我的英雄',
 '人间世',
 '新神榜：杨戬',
 '我的非凡父母',
 '哥，你好',
 '世间有她',
 '追光万里',
 '还是觉得你最好',
 '海的尽头是草原',
 '狼群',
 '妈妈！',
 '青蛙王国-极限运动',
 '追光',
 '钢铁意志',
 '平凡英雄']

In [7]:
feature_map

{'我的英雄': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]]),
 '人间世': array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 

#### Step
根据设定，在该环境下，决策需要包含两部分内容，第一部分是选定的6部影片，第二部分是这六部影片的具体排片占比。如下代码是`step`方法的测试代码：

In [8]:
selected_movie_name = ['我的英雄', '人间世', '新神榜：杨戬', '我的非凡父母', '哥，你好', '世间有她']
movie_show_ratio = np.array([0.2, 0.2, 0.2, 0.2, 0.1, 0.1])
feature_map, day_income, done, info = env.step((selected_movie_name, movie_show_ratio))

[INFO] [2025-05-09 23:23:58] ++++++++++++++++++++ [2022-10-02 周6] ++++++++++++++++++++++++
[INFO] [2025-05-09 23:23:58] 电影名称：        我的英雄        ，场次：20 ，单场均收入：659  ，总收入：13180  
[INFO] [2025-05-09 23:23:58] 电影名称：        人间世         ，场次：20 ，单场均收入：0    ，总收入：0      
[INFO] [2025-05-09 23:23:58] 电影名称：       新神榜：杨戬       ，场次：20 ，单场均收入：574  ，总收入：11480  
[INFO] [2025-05-09 23:23:58] 电影名称：       我的非凡父母       ，场次：20 ，单场均收入：8    ，总收入：160    
[INFO] [2025-05-09 23:23:58] 电影名称：        哥，你好        ，场次：10 ，单场均收入：484  ，总收入：4840   
[INFO] [2025-05-09 23:23:58] 电影名称：        世间有她        ，场次：10 ，单场均收入：127  ，总收入：1270   
[INFO] [2025-05-09 23:23:58] 截止目前的 总收入：30930，总效用：27129.1
[INFO] [2025-05-09 23:23:58] 明日可播放电影名称为：《我的英雄》, 《人间世》, 《新神榜：杨戬》, 《我的非凡父母》, 《哥，你好》, 《世间有她》, 《追光万里》, 《还是觉得你最好》, 《海的尽头是草原》, 《狼群》, 《妈妈！》, 《青蛙王国-极限运动》, 《追光》, 《钢铁意志》, 《平凡英雄》


In [9]:
feature_map

(['我的英雄',
  '人间世',
  '新神榜：杨戬',
  '我的非凡父母',
  '哥，你好',
  '世间有她',
  '追光万里',
  '还是觉得你最好',
  '海的尽头是草原',
  '狼群',
  '妈妈！',
  '青蛙王国-极限运动',
  '追光',
  '钢铁意志',
  '平凡英雄'],
 {'我的英雄': array([[0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
         [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
         [0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00,
          0.000000e+00, 0.000000e+00, 0.000000e+00, 0.000000e+00],
         [0.000000e+00, 0.000000e+00,

想要解决这个虚拟环境所假设的场景，需要考虑的内容包括：
（1）结构化数据与非结构化数据的结合
（2）探索与贪心的平衡问题
（3）不可观测数据的处理问题